In [ ]:
%pip install pandas
%pip install yfinance


# Importar biblioteca pandas
1. Baixar base de históricos
2. Calcular todos os MAs
3. Calcular o ranqueamento
4. Mostrar resultado do ranqueamento

In [ ]:
import pandas as pd
import json
import yfinance as yf
import datetime
import pytz


# Contém a lista de todas os simbolos da B3, obtidos do Yahoo finance via scrapping

In [ ]:
symbols_file = "data/symbols.json"
symbols_df = pd.read_json(symbols_file, orient='records')
symbols_df.head()


In [ ]:
symbols_df.shape[0]


# Gera dataframe, com fechamento diário, dos últimos 5 anos

In [ ]:
from calcEMA import run_calc_emas
dict_symbols = symbols_df.to_dict(orient='records')
list_historico = pd.DataFrame()
i = 0

for symbol in dict_symbols:
    print(symbol)

    # Baixa histórico de cotações dos últimos 5 anos do simbolo passado por parâmetro 
    hist = yf.download(
        tickers=symbol['symbol'],
        period="5y",
        interval="1d",
        auto_adjust=False,
        progress=False)
    hist['symbol'] = symbol['symbol']

    # Calcular o EMA + RSI de cada simbolo baixado no passo anterior
    hist = run_calc_emas(hist)
    hist = hist.last('1D')
    # Acumula na variável list_histórico os calculos de EMA + RSI do último dia de cotação
    list_historico = pd.concat([list_historico,  hist], axis=0)

    # Limitador de simbolos  à serem baixados.
    i += 1
    if i > 5:
        break


In [ ]:
list_historico

In [ ]:
tickers = pd.read_csv('./data/tickers_list_to_analisys.csv')
print(tickers.head())
tickers['symbol'] += '.SA'
tickers

In [ ]:
data = yf.download(list(tickers['symbol']), start='2023-01-01', threads=20, group_by='ticker' )
data

In [ ]:
symbols = []
for symbol, _ in data.columns:
    symbols.append(symbol)
symbols = list(set(symbols))

new_df = pd.DataFrame()
for s in symbols:
    aux = data[s].copy()
    aux['symbol'] = s
    new_df = pd.concat([new_df, aux], axis=0)

new_df.rename(columns={'Adj Close': 'adj_close', 'Close': 'close', 'High': 'high',
                'Low': 'low', 'Open': 'open', 'Close': 'close', 'Volume': 'volume'}, inplace=True)
new_df.index.name = 'date'
new_df['date_time'] = pd.to_datetime(new_df.index)
new_df['s_datetime'] = new_df.index.strftime('%Y%m%d%H%M')
new_df['date_import'] = datetime.datetime.now(tz=pytz.UTC)

new_df

In [ ]:
new_df.to_json(
        './data/ibov.json',
        orient='records',
        date_unit='s', date_format='epoch')